In [1]:
#hiddencell
import micropip
await micropip.install("seaborn")
# from pbl_tools import *

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [2]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [18]:
import pandas as pd 

train_org = pd.read_csv('D:\파이썬\대구\\train.csv',encoding='euc-kr') 
test_org = pd.read_csv('D:\파이썬\대구\\test.csv',encoding='euc-kr')


train_org = train_org.drop('열1',axis=1)
display(train_org.head())
display(test_org.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
12128,ACCIDENT_39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차
12129,ACCIDENT_39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차
12130,ACCIDENT_39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차
12131,ACCIDENT_39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차
12132,ACCIDENT_39608,2021-12-31 23,금요일,맑음,대구광역시 서구 비산동,단일로 - 지하차도(도로)내,건조,차대차


In [19]:
train_org.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')

In [28]:
# Specify the columns you want to extract
selected_columns = ['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수']

# Use square bracket notation to extract the specified columns
train_org = train_org[selected_columns]
train_org

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27471,ACCIDENT_27471,2020-12-31 20,목요일,맑음,대구광역시 서구 평리동,단일로 - 기타,건조,차대차,측면충돌,중앙선침범,이륜,남,68세,상해없음,이륜,남,40세,경상,0
27472,ACCIDENT_27472,2020-12-31 21,목요일,맑음,대구광역시 남구 대명동,단일로 - 기타,건조,차대차,후진중충돌,안전운전불이행,승용,남,35세,상해없음,이륜,남,44세,경상,0
27473,ACCIDENT_27473,2020-12-31 21,목요일,맑음,대구광역시 달서구 두류동,교차로 - 교차로안,건조,차대차,측면충돌,직진우회전진행방해,이륜,남,21세,경상,승용,남,36세,상해없음,0
27474,ACCIDENT_27474,2020-12-31 22,목요일,맑음,대구광역시 중구 삼덕동2가,교차로 - 교차로부근,건조,차대차,추돌,안전운전불이행,승용,여,54세,부상신고,이륜,남,22세,경상,0


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [59]:
display(f"train : {train_org.iloc[0]['사고일시']} ~ {train_org.iloc[-1]['사고일시']}")
display(f"test : {test_org.iloc[0]['사고일시']} ~ {test_org.iloc[-1]['사고일시']}")     

'train : 2019-01-01 00 ~ 2020-12-31 23'

'test : 2021-01-01 09 ~ 2021-12-31 23'

## **예측 대상 확인하기**

sample_submission.csv를 로드하여 출력 형식을 확인해 봄으로서, 
대회 페이지 규칙에 기술되어 있는, 예측 대상이 무엇인지 확인해 봅시다.

최종적으로 예측해야 하는 값은 'ECLO'(인명피해 심각도) 값입니다. 

그러나, train데이터 컬럼에 있는 '사망자수', '중상자수', '경상자수', '부상자수'를 각각 예측하여 ECLO 예측값을 구할 수도 있습니다. 즉, ECLO 값을 예측하는 단일 모델로 접근할 수도,  각각을 별도로 예측하는 모델로 접근할 수도 있습니다. 

In [60]:
sample_submission = pd.read_csv("D:\파이썬\대구\sample_submission.csv")
sample_submission.head()

,ID,ECLO
0,ACCIDENT_27476,0
1,ACCIDENT_27477,0
2,ACCIDENT_27478,0
3,ACCIDENT_27479,0
4,ACCIDENT_27480,0


## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [61]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다 
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 

# 해당 과정을 test_x에 대해서도 반복해줍니다 
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사망자수', '연', '월',\n       '일', '시간'],\n      dtype='object')"

## **파생 변수 생성 2 : 공간(위치) 정보 생성** 

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [62]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사망자수', '연', '월', '일', '시간',\n       '도시', '구', '동'],\n      dtype='object')"

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [63]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사망자수', '연', '월', '일', '시간', '도시',\n       '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다 

In [64]:
display(train_df.head())
display(test_df.head())

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,ECLO,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,...,5,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,...,3,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,...,3,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,...,5,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,...,3,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


,ID,요일,기상상태,노면상태,사고유형,사망자수,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_27476,금요일,맑음,건조,차대차,0.010304,2021,1,1,9,대구광역시,달성군,유가읍,단일로,기타
1,ACCIDENT_27477,금요일,맑음,건조,차대차,0.002933,2021,1,1,9,대구광역시,서구,평리동,교차로,교차로안
2,ACCIDENT_27478,금요일,맑음,건조,차대차,0.002284,2021,1,1,13,대구광역시,달서구,상인동,단일로,기타
3,ACCIDENT_27479,금요일,맑음,건조,차대차,0.004019,2021,1,1,13,대구광역시,북구,산격동,교차로,교차로안
4,ACCIDENT_27480,금요일,맑음,건조,차대차,0.002933,2021,1,1,14,대구광역시,서구,평리동,교차로,교차로횡단보도내


## **train, test 데이터의 독립변수, 종속 변수 분리**

모델 추론 시점에서 획득할 수 있는 데이터는 학습 시점에서 활용할 수 있는 데이터와 다를 수 있습니다  
그렇기에 본 baseline에서는 test 시점에서만 활용할 수 있는 데이터들(test의 column들)만 사용하여 train 데이터를 구성했습니다 

train_df 중에서 test_x가 가진 column들만 추출하여 train_x(독립변수)를 구성하고, train_df 중에서 예측 목표인 EPDO를 추출하여 train_y(종속변수)를 구성합니다

In [66]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [67]:
train_y

0        5
1        3
2        3
3        5
4        3
        ..
27471    3
27472    3
27473    3
27474    4
27475    3
Name: ECLO, Length: 27476, dtype: int64

## **범주형(Categorical) 변수, 수치형 변수로 변환하기**

모델 학습을 위해 train_x의 문자열 타입의 컬럼들을 추출하고, LabelEncoder를 활용하여 이 컬럼들을 모두 수치형 변수로 변환해 보겠습니다

In [68]:
from sklearn.preprocessing import LabelEncoder

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_x[i]) 
    train_x[i]=le.transform(train_x[i])
    
    for case in np.unique(test_x[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test_x[i]=le.transform(test_x[i])
    
display(train_x.head())
display(test_x.head())

['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

,요일,기상상태,노면상태,사고유형,사망자수,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,6,2,0,0,0,2019,1,1,0,0,7,40,2,5
1,6,5,0,0,0,2019,1,1,0,0,1,4,2,5
2,6,2,0,0,0,2019,1,1,1,0,6,64,2,5
3,6,2,0,1,0,2019,1,1,2,0,4,77,2,5
4,6,2,0,1,0,2019,1,1,4,0,3,127,2,5


,요일,기상상태,노면상태,사고유형,사망자수,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,0,2,0,1,0.010304,2021,1,1,9,0,2,144,2,5
1,0,2,0,1,0.002933,2021,1,1,9,0,5,180,0,3
2,0,2,0,1,0.002284,2021,1,1,13,0,1,101,2,5
3,0,2,0,1,0.004019,2021,1,1,13,0,4,92,0,3
4,0,2,0,1,0.002933,2021,1,1,14,0,5,180,0,4


## **Model Train & Prediction** 

전체 학습 데이터에 대하여 학습후 최종 예측을 합니다. 여기서는 DecisionTreeRegressor 모델로 학습하고 예측하는 예시 코드입니다.

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

# Function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# Function to calculate RMSLE
def calculate_rmsle(y_true, y_pred):
    log_diff = np.log1p(y_pred) - np.log1p(y_true)
    rmsle = np.sqrt(np.mean(np.square(log_diff)))
    return rmsle

(16485, 14) (5495, 14) (5496, 14) (16485,) (5495,) (5496,)


In [38]:
# Initialize and train the model

# GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"GradientBoostingRegressor RMSE on Test Set: {rmse}")
print(f"GradientBoostingRegressor RMSLE on Test Set: {rmsle}")


# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
gb_model = DecisionTreeRegressor()
gb_model.fit(X_train, y_train)
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"DecisionTreeRegressor RMSE on Test Set: {rmse}")
print(f"DecisionTreeRegressor RMSLE on Test Set: {rmsle}")

# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
gb_model = RandomForestRegressor()
gb_model.fit(X_train, y_train)
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"RandomForestRegressor RMSE on Test Set: {rmse}")
print(f"RandomForestRegressor RMSLE on Test Set: {rmsle}")
x

# LGBMRegressor
from lightgbm.sklearn import LGBMRegressor   
from lightgbm import LGBMRegressor, early_stopping
gb_model = LGBMRegressor(num_leaves=11, learning_rate=0.05, n_estimators=100, verbose=-1)
gb_model.fit(train_x, train_y, eval_set=[(X_val, y_val)], eval_metric='rmse', callbacks=[early_stopping(stopping_rounds=10)])
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"LGBMRegressor RMSE on Test Set: {rmse}")
print(f"LGBMRegressor RMSLE on Test Set: {rmsle}")



# 0.4518745213
# RandomForestRegressor RMSE on Test Set: 0.08216978714119436
# RandomForestRegressor RMSLE on Test Set: 0.011193171663530828
# RandomForestRegressor RMSE on Test Set: 0.06350783170158267
# RandomForestRegressor RMSLE on Test Set: 0.009891285397262447

GradientBoostingRegressor RMSE on Test Set: 0.09269694541282411
GradientBoostingRegressor RMSLE on Test Set: 0.06352509954731203
DecisionTreeRegressor RMSE on Test Set: 0.139530299621167
DecisionTreeRegressor RMSLE on Test Set: 0.09467623063439604
RandomForestRegressor RMSE on Test Set: 0.09841101682049229
RandomForestRegressor RMSLE on Test Set: 0.07028793357618765
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.0907789	valid_0's l2: 0.00824081
LGBMRegressor RMSE on Test Set: 0.08733648808422793
LGBMRegressor RMSLE on Test Set: 0.05893495427411047


In [71]:
train_y

0        5
1        3
2        3
3        5
4        3
        ..
27471    3
27472    3
27473    3
27474    4
27475    3
Name: ECLO, Length: 27476, dtype: int64

In [46]:
# LGBMRegressor
from lightgbm.sklearn import LGBMRegressor   
from lightgbm import LGBMRegressor, early_stopping
gb_model = LGBMRegressor(num_leaves=11, learning_rate=0.05, n_estimators=100, verbose=-1)
gb_model.fit(train_x, train_y, eval_set=[(X_val, y_val)], eval_metric='rmse', callbacks=[early_stopping(stopping_rounds=10)])
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"LGBMRegressor RMSE on Test Set: {rmse}")
print(f"LGBMRegressor RMSLE on Test Set: {rmsle}")


prediction = gb_model.predict(test_x)


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.0907789	valid_0's l2: 0.00824081
LGBMRegressor RMSE on Test Set: 0.08733648808422793
LGBMRegressor RMSLE on Test Set: 0.05893495427411047


In [47]:
# 붙여넣기 
test_org['사망자수'] = prediction
test_org

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사망자수
0,ACCIDENT_27476,2021-01-01 09,금요일,맑음,대구광역시 달성군 유가읍,단일로 - 기타,건조,차대차,0.010304
1,ACCIDENT_27477,2021-01-01 09,금요일,맑음,대구광역시 서구 평리동,교차로 - 교차로안,건조,차대차,0.002933
2,ACCIDENT_27478,2021-01-01 13,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,0.002284
3,ACCIDENT_27479,2021-01-01 13,금요일,맑음,대구광역시 북구 산격동,교차로 - 교차로안,건조,차대차,0.004019
4,ACCIDENT_27480,2021-01-01 14,금요일,맑음,대구광역시 서구 평리동,교차로 - 교차로횡단보도내,건조,차대차,0.002933
...,...,...,...,...,...,...,...,...,...
12128,ACCIDENT_39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,0.002592
12129,ACCIDENT_39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,0.001943
12130,ACCIDENT_39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,0.006338
12131,ACCIDENT_39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,0.006338


In [55]:

train_org = pd.read_csv('D:\파이썬\대구\\train.csv',encoding='euc-kr') 
train_org = train_org.drop('열1',axis=1)
train_org

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27471,ACCIDENT_27471,2020-12-31 20,목요일,맑음,대구광역시 서구 평리동,단일로 - 기타,건조,차대차,측면충돌,중앙선침범,...,상해없음,이륜,남,40세,경상,0,0,1,0,3
27472,ACCIDENT_27472,2020-12-31 21,목요일,맑음,대구광역시 남구 대명동,단일로 - 기타,건조,차대차,후진중충돌,안전운전불이행,...,상해없음,이륜,남,44세,경상,0,0,1,0,3
27473,ACCIDENT_27473,2020-12-31 21,목요일,맑음,대구광역시 달서구 두류동,교차로 - 교차로안,건조,차대차,측면충돌,직진우회전진행방해,...,경상,승용,남,36세,상해없음,0,0,1,0,3
27474,ACCIDENT_27474,2020-12-31 22,목요일,맑음,대구광역시 중구 삼덕동2가,교차로 - 교차로부근,건조,차대차,추돌,안전운전불이행,...,부상신고,이륜,남,22세,경상,0,0,1,1,4


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

# Function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# Function to calculate RMSLE
def calculate_rmsle(y_true, y_pred):
    log_diff = np.log1p(y_pred) - np.log1p(y_true)
    rmsle = np.sqrt(np.mean(np.square(log_diff)))
    return rmsle

(16485, 13) (5495, 13) (5496, 13) (16485,) (5495,) (5496,)


In [70]:
# LGBMRegressor
from lightgbm.sklearn import LGBMRegressor   
from lightgbm import LGBMRegressor, early_stopping
gb_model = LGBMRegressor(num_leaves=11, learning_rate=0.05, n_estimators=100, verbose=-1)
gb_model.fit(train_x, train_y, eval_set=[(X_val, y_val)], eval_metric='rmse', callbacks=[early_stopping(stopping_rounds=10)])
prediction = gb_model.predict(X_test)
rmse = calculate_rmse(y_test, prediction)
rmsle = calculate_rmsle(y_test, prediction)
print(f"LGBMRegressor RMSE on Test Set: {rmse}")
print(f"LGBMRegressor RMSLE on Test Set: {rmsle}")


prediction = gb_model.predict(test_x)


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 3.03474	valid_0's l2: 9.20962
LGBMRegressor RMSE on Test Set: 3.0707315822772014
LGBMRegressor RMSLE on Test Set: 0.45925620147466434


## **Submission 양식 확인**

sample_submission.csv 화일 데이터(sample_submission)를 그대로 복사한 후, 
양식의 'ECLO' 컬럼에 test_x에 대한 ECLO(y) 예측값을 입력합니다 

In [ ]:
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = prediction
baseline_submission 

,ID,ECLO
0,ACCIDENT_27476,4.996051
1,ACCIDENT_27477,4.868250
2,ACCIDENT_27478,4.695849
3,ACCIDENT_27479,4.834743
4,ACCIDENT_27480,4.538443
...,...,...
12128,ACCIDENT_39604,4.638588
12129,ACCIDENT_39605,4.565727
12130,ACCIDENT_39606,4.918780
12131,ACCIDENT_39607,4.580573


## **답안지 저장 및 제출하기**

In [ ]:
baseline_submission.to_csv('LGBMRegressor.csv', index=False)